In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

from numba import jit

import requests

### References
- December 2008 S&P 100 list: https://en.wikipedia.org/w/index.php?title=S%26P_100&oldid=255457499 (first version of the page with the actual list)
- S&P 100 Additions and Deletions 2004 to 2016: https://www.newlowobserver.com/2017/09/sp-100-additions-and-deletions-2004-to-2017
- FTSE100 December 2006 list: https://en.wikipedia.org/w/index.php?title=FTSE_100_Index&oldid=97533049
- December 2006 HANG SENG lsit: https://en.wikipedia.org/w/index.php?title=Hang_Seng_Index&oldid=97178424

In [2]:
def get_index_tickers(url, table_index, ticker_column_name='Symbol'):

    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    response = requests.get(url, headers=headers, verify=True)

    response.raise_for_status() 

    payload = pd.read_html(response.text)

    table = payload[table_index]
    tickers = table[ticker_column_name].tolist()

    tickers = [str(t).split(' ')[0] for t in tickers]

    return tickers
    


### S&P 100 Data

In [3]:
wikipedia_historical_page_dec_2008 = "https://en.wikipedia.org/w/index.php?title=S%26P_100&oldid=255457499"
sp100_2008_indices = get_index_tickers(wikipedia_historical_page_dec_2008, 0)

Now, we have to reverse engineer the S&P stocks list in 2006 based on the information contained in the reference: https://www.newlowobserver.com/2017/09/sp-100-additions-and-deletions-2004-to-2017.

In [4]:
# Changes in 2008
added_tickers_2008 = [
    'UNH', 'PM', 'NOV', 'QCOM', 'OXY', 'WYE', 'GILD', 'LOW', 'DVN',
    'LMT', 'WAG', 'SGP', 'NKE', 'AMZN', 'COST'
]
dropped_tickers_2008 = [
    'HET', 'ATI', 'ROK', 'CCU', 'LEH', 'BUD', 'IP', 'CBS', 'EP',
    'AES', 'HIG', 'AIG', 'CI', 'MER', 'WB'  
]

# Changes in 2007
added_tickers_2007 = [
    'CVS', 'KFT', 'AAPL', 'BK'
]
dropped_tickers_2007 = [
    'BDK', 'EK', 'MEDI', 'CSC'
]

# Changes in 2006
added_tickers_2006 = [
    'WB', 'RF', 'GOOG', 'COP'  
]
dropped_tickers_2006 = [
    'UIS', 'RSH', 'HCA', 'COF'
]

In [5]:
sp100_set = set(sp100_2008_indices)

n_periods_rollback = 3

tickers_to_add = [dropped_tickers_2006, dropped_tickers_2007, dropped_tickers_2008]
tickers_to_remove = [added_tickers_2006, added_tickers_2007, added_tickers_2008]

for i in range(n_periods_rollback):
    sp100_set.update(tickers_to_add[i])
    sp100_set.difference_update(tickers_to_remove[i])

Now we source from `yfinance` the stock history for this stock and we save it as a csv.

In [6]:
tickers_string = " ".join(list(sp100_set))

In [7]:
sp100_tickers = yf.Tickers(tickers_string)
sp100_hist = sp100_tickers.history(start= "2004-01-01", period= "2y")

[*********************100%***********************]  100 of 100 completed

25 Failed downloads:
['MA', 'S', 'BHI', 'BUD', 'CSC', 'HET', 'MER', 'EMC', 'RSH', 'MEDI', 'DOW', 'SLE', 'WB', 'HCA', 'COV', 'DELL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1072933200, endDate = 1136091600")')
['BNI', 'LEH', 'TYC', 'NYX', 'EK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01)')
['UTX', 'RTN', 'AVP', 'CBS']: YFTzMissingError('possibly delisted; no timezone found')


In [52]:
def clean_ticker_history(ticker_hist, threshold = 450):
    extract_close_set_hist_range = ticker_hist[("Close",)]["2004-01-01":"2006-12-31"]

    # we only get the stocks for which there are no more than a certain number of nan values
    cleaned_history = extract_close_set_hist_range.dropna(axis = 1, thresh = threshold)
    cleaned_history = cleaned_history.fillna(method = "ffill")

    # we check the remaining number of null values, just to be sure
    print(f"The remaining number of NaN values is {cleaned_history.isna().sum().sum()}")

    return cleaned_history

In [53]:
sp100_2004_2006 = clean_ticker_history(sp100_hist)

The remaining number of NaN values is 0


In [54]:
sp100_2004_2006.shape

(504, 75)

In [55]:
sp100_2004_2006.to_csv("C:\\Users\\Saverio\\Documents\\GitHub\\python_defi_project\\data\\sp100.csv")

### FTSE

In [30]:
ftse_100_tickers_2006 = [
    'III.L', 'AL.L', 'AB.L', 'AVZ.L', 'AAL.L', 'ANTO.L', 'ABF.L', 'AZN.L', 'AV.L',
    'BA.L', 'BARC.L', 'BG.L', 'BHP.L', 'BP.L', 'BB.L', 'BXB.L', 'BAY.L', 'BATS.L',
    'BLND.L', 'SKY.L', 'BT-A.L', 'CBRY.L', 'CNE.L', 'CPI.L', 'CCL.L', 'CNA.L',
    'CPG.L', 'CS.L', 'DGE.L', 'DRX.L', 'DSGI.L', 'ETI.L', 'EXPN.L', 'FP.L', 'GLH.L',
    'GSK.L', 'HMSO.L', 'HNS.L', 'HBOS.L', 'HOME.L', 'HSBA.L', 'IAP.L', 'ICI.L',
    'IMT.L', 'IHG.L', 'IPR.L', 'ITV.L', 'SBRY.L', 'JMAT.L', 'KAZ.L', 'KEL.L',
    'KGF.L', 'LAND.L', 'LGEN.L', 'LII.L', 'LLOY.L', 'LMI.L', 'EMG.L', 'MKS.L',
    'MRW.L', 'NG.L', 'NXT.L', 'NRK.L', 'OML.L', 'PSON.L', 'PSN.L', 'PRU.L', 'RB.L',
    'REL.L', 'RSL.L', 'RTR.L', 'REX.L', 'RIO.L', 'RR.L', 'RSA.L', 'RBS.L', 'RDSA.L',
    'SAB.L', 'SGE.L', 'SCTN.L', 'SSE.L', 'SPW.L', 'SVT.L', 'SHP.L', 'SLOU.L',
    'SN.L', 'SMIN.L', 'STAN.L', 'SL.L', 'TATE.L', 'TSCO.L', 'ULVR.L', 'UU.L',
    'VED.L', 'VOD.L', 'WTB.L', 'WOS.L', 'WPP.L', 'XTA.L', 'YELL.L'
]

In [32]:
ftse_tickers = yf.Tickers(ftse_100_tickers_2006)
ftse_hist = ftse_tickers.history(start= "2004-01-01", period= "2y")

[*********************100%***********************]  100 of 100 completed

43 Failed downloads:
['SPW.L', 'IMT.L', 'KEL.L', 'GLH.L', 'YELL.L', 'NRK.L', 'SAB.L', 'HBOS.L', 'HNS.L', 'ICI.L', 'DSGI.L', 'AVZ.L', 'IAP.L', 'SLOU.L', 'LII.L', 'RTR.L', 'CBRY.L', 'REX.L', 'SCTN.L', 'ETI.L', 'AL.L', 'BG.L', 'WOS.L', 'SL.L', 'BB.L', 'IPR.L']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01)')
['LMI.L', 'RBS.L', 'RSA.L', 'RDSA.L', 'MRW.L', 'CS.L', 'RB.L', 'AB.L', 'KAZ.L', 'BXB.L']: YFTzMissingError('possibly delisted; no timezone found')
['HOME.L', 'FP.L', 'EXPN.L', 'BAY.L']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1072915200, endDate = 1136073600")')
['RSL.L', 'SKY.L', 'OML.L']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1072933200, endDate = 1136091600"

In [56]:
# There are still some nan values in the first row, 3 in total, so we remove them
ftse_hist_2006 = clean_ticker_history(ftse_hist)

The remaining number of NaN values is 3


In [57]:
ftse_hist_2006 = ftse_hist_2006.dropna()

In [58]:
ftse_hist_2006.shape

(521, 56)

In [59]:
ftse_hist_2006.to_csv("C:\\Users\\Saverio\\Documents\\GitHub\\python_defi_project\\data\\ftse100.csv")

### HANG SENG Index

In [43]:
hang_seng_tickers_2006_era = [
    # Finance
    '0005.HK', '0011.HK', '0023.HK', '2388.HK',
    # Utilities
    '0002.HK', '0003.HK', '0006.HK',
    # Property
    '0001.HK', '0012.HK', '0016.HK', '0083.HK', '0101.HK',
    # Commercial & Industrial
    '0004.HK', '0008.HK', '0013.HK', '0017.HK', '0019.HK', '0066.HK',
    '0144.HK', '0179.HK', '0267.HK', '0291.HK', '0293.HK', '0330.HK',
    '0494.HK', '0551.HK', '0762.HK', '0883.HK', '0906.HK', '0941.HK',
    '0992.HK', '1038.HK', '1199.HK'
]

In [44]:
hangseng_tickers = yf.Tickers(hang_seng_tickers_2006_era)
hangseng_hist = hangseng_tickers.history(start= "2004-01-01", period= "2y")

[*********************100%***********************]  33 of 33 completed

3 Failed downloads:
['0906.HK', '0494.HK']: YFTzMissingError('possibly delisted; no timezone found')
['0013.HK']: YFPricesMissingError('possibly delisted; no price data found  (1d 2004-01-01 -> 2006-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1072886400, endDate = 1136044800")')


In [60]:
hangseng_hist_2006 = clean_ticker_history(hangseng_hist)

The remaining number of NaN values is 54


In [65]:
# The NaN values correspond to the missing entries of 0883 HK
hangseng_hist_2006["0883.HK"].isna().sum()

# as the data will then be converted to pct_changes and there are still at least 450 non-na observations, we don't perform any modifications

54

In [66]:
hangseng_hist_2006.shape

(509, 30)

In [67]:
hangseng_hist_2006.to_csv("C:\\Users\\Saverio\\Documents\\GitHub\\python_defi_project\\data\\hanseng.csv")